In [59]:
import pandas as pd
import numpy as np
import os

from tqdm import tqdm
from unicodedata import normalize

### Analizo Data de Tribunales y Cortes de Apelaciones

In [60]:
path_processed = "../data/processed/pjud"
path_censo = "../data/processed/censo"

In [61]:
df_regiones = pd.read_feather(f"{path_processed}/TerminoRol_feather")
df_tribunales = pd.read_feather(f"{path_processed}/ListadoTribunales_feather")
df_censo = pd.read_feather(f"{path_censo}/Censo2017_feather")

In [62]:
# Extraigo Cortes de Apelaciones asociadas a Juzgados desde este DataSet
data_cortes_apelaciones = pd.unique(df_regiones[['CORTE','TRIBUNAL']].values.ravel())
cortes_tribunales = []

for data in range(len(data_cortes_apelaciones)):  
    if not data_cortes_apelaciones[data].find('C.A.') == -1:
        corte_apelacion = data_cortes_apelaciones[data]
    else:
        tribunal = data_cortes_apelaciones[data]
        if tribunal.find("TRIBUNAL") != -1:
            separa_ciudad = tribunal.split("PENAL ")
        else:
            separa_ciudad = tribunal.split("GARANTIA ")
        ciudad = separa_ciudad[1]
        cortes_tribunales.append([corte_apelacion,ciudad])

In [63]:
lista_cortes = []

for trib in df_tribunales.index:
    for indice in range(len(cortes_tribunales)):
        if df_tribunales['ASIENTO'][trib] == cortes_tribunales[indice][1]:
            corte = cortes_tribunales[indice][0]
            lista_cortes.append(corte)
            break

df_tribunales['CORTE'] = lista_cortes

In [64]:
df_tribunales

,index,REGION,TRIBUNAL,ASIENTO,COMUNA,CORTE
0,0,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,HUARA,C.A. DE IQUIQUE
1,1,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,CAMIÑA,C.A. DE IQUIQUE
2,2,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,COLCHANE,C.A. DE IQUIQUE
3,3,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,IQUIQUE,C.A. DE IQUIQUE
4,4,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,POZO ALMONTE,C.A. DE IQUIQUE
...,...,...,...,...,...,...
687,341,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,PRIMAVERA,C.A. DE PUNTA ARENAS
688,342,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,TIMAUKEL,C.A. DE PUNTA ARENAS
689,343,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,CABO DE HORNOS,C.A. DE PUNTA ARENAS
690,344,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,ANTARTICA,C.A. DE PUNTA ARENAS


In [65]:
df_censo.columns

Index(['index', 'ORDEN', 'NOMBRE REGIÓN', 'CÓDIGO REGIÓN', 'NOMBRE PROVINCIA',
       'CÓDIGO PROVINCIA', 'NOMBRE COMUNA', 'CÓDIGO COMUNA', 'EDAD',
       'TOTAL POBLACIÓN EFECTIVAMENTE CENSADA', 'HOMBRES ', 'MUJERES',
       'TOTAL ÁREA URBANA', 'HOMBRES ÁREA URBANA ', 'MUJERES ÁREA URBANA',
       'TOTAL ÁREA RURAL', 'HOMBRES ÁREA RURAL', 'MUJERES ÁREA RURAL'],
      dtype='object')

In [66]:
poblacion = []

for trib in df_tribunales.index:
    for indice in df_censo.index:
        if df_tribunales['COMUNA'][trib] == df_censo['NOMBRE COMUNA'][indice]:
            censado = df_censo['TOTAL POBLACIÓN EFECTIVAMENTE CENSADA'][indice]
            hombres = df_censo['HOMBRES '][indice]
            mujeres = df_censo['MUJERES'][indice]
            urbana = df_censo['TOTAL ÁREA URBANA'][indice]
            rural = df_censo['TOTAL ÁREA RURAL'][indice]
            poblacion.append([censado, hombres, mujeres, urbana, rural])
            break

df_poblacion = pd.DataFrame(poblacion, columns = ['POBLACION', 'HOMBRES', 'MUJERES', 'URBANO', 'RURAL'])

In [67]:
df_poblacion

,POBLACION,HOMBRES,MUJERES,URBANO,RURAL
0,2730,1501,1229,1109,1621
1,1250,657,593,0,1250
2,1728,995,733,0,1728
3,191468,94897,96571,189065,2403
4,15711,8987,6724,10095,5616
...,...,...,...,...,...
687,1158,930,228,0,1158
688,405,343,62,0,405
689,2063,1195,868,1874,189
690,138,126,12,0,138


In [68]:
df_tribunales_poblacion = pd.concat([df_tribunales, df_poblacion.reindex(df_tribunales.index)], axis=1)

In [69]:
df_tribunales_poblacion

,index,REGION,TRIBUNAL,ASIENTO,COMUNA,CORTE,POBLACION,HOMBRES,MUJERES,URBANO,RURAL
0,0,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,HUARA,C.A. DE IQUIQUE,2730,1501,1229,1109,1621
1,1,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,CAMIÑA,C.A. DE IQUIQUE,1250,657,593,0,1250
2,2,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,COLCHANE,C.A. DE IQUIQUE,1728,995,733,0,1728
3,3,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,IQUIQUE,C.A. DE IQUIQUE,191468,94897,96571,189065,2403
4,4,REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,POZO ALMONTE,C.A. DE IQUIQUE,15711,8987,6724,10095,5616
...,...,...,...,...,...,...,...,...,...,...,...
687,341,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,PRIMAVERA,C.A. DE PUNTA ARENAS,1158,930,228,0,1158
688,342,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA PORVENIR,PORVENIR,TIMAUKEL,C.A. DE PUNTA ARENAS,405,343,62,0,405
689,343,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,CABO DE HORNOS,C.A. DE PUNTA ARENAS,2063,1195,868,1874,189
690,344,REGION DE MAGALLANES Y ANTARTICA CHILENA,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,CABO DE HORNOS,ANTARTICA,C.A. DE PUNTA ARENAS,138,126,12,0,138


## CARGA DATA A ARCHIVOS FUTHER

In [70]:
# Reset el index para realizar feather
df_tribunales.reset_index(inplace = True)
df_tribunales_poblacion.reset_index(inplace = True)
# Directorio donde se guardaran archivos feather
os.makedirs(path_processed, exist_ok = True) 
df_tribunales.to_feather(f'{path_processed}/ListadoTribunalesyCortes_feather')
df_tribunales_poblacion.to_feather(f'{path_processed}/Poblacion_Tribunales_feather')